# Differentiable bitonic sort

[Bitonic sorts](https://en.wikipedia.org/wiki/Bitonic_sorter) allow creation of sorting networks with a sequence of fixed conditional swapping operations executed in parallel. A sorting network implements  a map from $\mathbb{R}^n \rightarrow \mathbb{R}^n$, where $n=2^k$ (sorting networks for non-power-of-2 sizes are possible but not trickier).

<img src="BitonicSort1.svg.png">

*[Image: from Wikipedia, by user Bitonic, CC0](https://en.wikipedia.org/wiki/Bitonic_sorter#/media/File:BitonicSort1.svg)*

The sorting network for $n=2^k$ elements has $\frac{k(k-1)}{2}$ "layers" where parallel compare-and-swap operations are used to rearrange a $k$ element vector into sorted order.

### Differentiable compare-and-swap

If we define the `softmax(a,b)` function (not the traditional "softmax" used for classification!) as the continuous approximation to the `max(a,b)` function:

$$\text{softmax}(a,b) = \log(e^a + e^b) \approx \max(a,b).$$

We can then fairly obviously write `softmin(a,b)` as:

$$\text{softmin}(a,b) = -\log(e^{-a} + e^{-b}) \approx \min(a,b).$$

These functions obviously aren't equal to max and min, but are relatively close, and differentiable. Note that we now have a differentiable compare-and-swap operation:

$$\text{high} = \text{softmax}(a,b), \text{low} = \text{softmin}(a,b), \text{where } \text{low}\leq \text{high}$$

## Differentiable sorting

For each layer in the sorting network, we can split all of the pairwise comparison-and-swaps into left-hand and right-hand sides which can be done simultaneously. We can any write function that selects the relevant elements of the vector as a multiply with a binary matrix.

For each layer, we can derive two binary matrices $L \in \mathbb{R}^{n \times \frac{n}{2}}$ and $R \in \mathbb{R}^{n \times \frac{n}{2}}$ which select the elements to be compared for the left and right hands respectively. This will result in the comparison between two $\frac{k}{2}$ length vectors. We can also derive two matrices $L' \in \mathbb{R}^{\frac{n}{2} \times n}$ and $R' \in \mathbb{R}^{\frac{n}{2} \times n}$ which put the results of the compare-and-swap operation back into the right positions.

Then, each layer $i$ of the sorting process is just:
$${\bf x}_{i+1} = L'_i[\text{softmin}(L_i{\bf x_i}, R_i{\bf x_i})] + R'_i[\text{softmax}(L_i{\bf x_i}, R_i{\bf x_i})]$$
$$ = L'_i\left(-\log\left(e^{-L_i{\bf x}_i} + e^{-R_i{\bf x}_i}\right)\right) +  R'_i\left(\log\left(e^{L_i{\bf x}_i} + e^{R_i{\bf x}_i}\right)\right)$$
which is clearly differentiable (though not very numerically stable -- the usable range of elements $x$ is quite limited in single float precision).

All that remains is to compute the matrices $L_i, R_i, L'_i, R'_i$ for each of the layers of the network. 

This process is excessively computation heavy, but easy to compute. We could also simplify this into two matrix multiplies, at the cost of a vector split and join in the middle (see the `woven` form later in this text). 

## Example

To sort four elements, we have a network like:

    0  1  2  3  
    ┕>>┙  │  │  
    │  │  ┕<<┙  
    ┕>>>>>┙  │  
    │  │  │  │  
    ┕>>┙  │  │  
    │  │  ┕>>┙  
    
This is equivalent to: 

    x[0], x[1] = cswap(x[0], x[1])
    x[3], x[2] = cswap(x[2], x[3])
    x[0], x[2] = cswap(x[0], x[2])
    x[0], x[1] = cswap(x[0], x[1])
    x[2], x[3] = cswap(x[2], x[3])
    
where `cswap(a,b) = (min(a,b), max(a,b))`

Replacing the indexing with matrix multiplies and `cswap` with a `softcswap = (softmin(a,b), softmax(a,b))` we then have the differentiable form.



# Test functions

In [1]:
from bitonic_tests import bitonic_network, pretty_bitonic_network

def neat_vec(n):
    # print a vector neatly    
    return "\t".join([f"{x:.2f}" for x in n])

# this should match the diagram at the top of the notebook
bitonic_network(16)

 0>1	 2<3	 4>5	 6<7	 8>9	10<11	12>13	14<15	
----------------------------------------------------------------
 0>2	 1>3	 4<6	 5<7	 8>10	 9>11	12<14	13<15	
 0>1	 2>3	 4<5	 6<7	 8>9	10>11	12<13	14<15	
----------------------------------------------------------------
 0>4	 1>5	 2>6	 3>7	 8<12	 9<13	10<14	11<15	
 0>2	 1>3	 4>6	 5>7	 8<10	 9<11	12<14	13<15	
 0>1	 2>3	 4>5	 6>7	 8<9	10<11	12<13	14<15	
----------------------------------------------------------------
 0>8	 1>9	 2>10	 3>11	 4>12	 5>13	 6>14	 7>15	
 0>4	 1>5	 2>6	 3>7	 8>12	 9>13	10>14	11>15	
 0>2	 1>3	 4>6	 5>7	 8>10	 9>11	12>14	13>15	
 0>1	 2>3	 4>5	 6>7	 8>9	10>11	12>13	14>15	
----------------------------------------------------------------


In [2]:
pretty_bitonic_network(16)

0  1  2  3  4  5  6  7  8  9  10 11 12 13 14 15 
+>>+  │  │  │  │  │  │  │  │  │  │  │  │  │  │  
│  │  +<<+  │  │  │  │  │  │  │  │  │  │  │  │  
│  │  │  │  +>>+  │  │  │  │  │  │  │  │  │  │  
│  │  │  │  │  │  +<<+  │  │  │  │  │  │  │  │  
│  │  │  │  │  │  │  │  +>>+  │  │  │  │  │  │  
│  │  │  │  │  │  │  │  │  │  +<<+  │  │  │  │  
│  │  │  │  │  │  │  │  │  │  │  │  +>>+  │  │  
│  │  │  │  │  │  │  │  │  │  │  │  │  │  +<<+  
+>>>>>+  │  │  │  │  │  │  │  │  │  │  │  │  │  
│  │  │  │  │  │  │  │  │  │  │  │  │  │  │  │  
│  │  │  │  +<<<<<+  │  │  │  │  │  │  │  │  │  
│  │  │  │  │  │  │  │  │  │  │  │  │  │  │  │  
│  │  │  │  │  │  │  │  +>>>>>+  │  │  │  │  │  
│  │  │  │  │  │  │  │  │  │  │  │  │  │  │  │  
│  │  │  │  │  │  │  │  │  │  │  │  +<<<<<+  │  
│  │  │  │  │  │  │  │  │  │  │  │  │  │  │  │  
+>>+  │  │  │  │  │  │  │  │  │  │  │  │  │  │  
│  │  +>>+  │  │  │  │  │  │  │  │  │  │  │  │  
│  │  │  │  +<<+  │  │  │  │  │  │  │  │  │  │  
│  │  │  │  │  │  +<

# Vectorised functions

## Testing

In [3]:
# Test sorting
import autograd.numpy as np # we can use plain numpy as well (but can't take grad!)


from differentiable_sorting import bitonic_matrices, diff_bisort, diff_argsort, softmax, softmin, softcswap, bisort
matrices = bitonic_matrices(8)

for i in range(10):
    # these should all be in sorted order
    test = np.random.randint(0, 200, 8)
    print(bisort(matrices, test))    

[  2.  10.  11. 101. 108. 123. 133. 187.]
[ 53.  67.  94. 106. 131. 154. 187. 196.]
[  3.  24.  36. 142. 148. 155. 168. 194.]
[  5.  15.  30.  30.  30.  79. 111. 176.]
[ 17.  36.  42.  83. 101. 142. 160. 165.]
[ 84.  91.  94. 112. 116. 136. 179. 194.]
[ 15.  42.  57.  65.  87. 109. 168. 184.]
[ 19.  25.  52.  60.  65.  71. 132. 188.]
[  3.  14.  32.  44.  73.  88. 141. 193.]
[ 42.  86.  94.  95. 103. 107. 116. 137.]


In [4]:
for i in range(1, 11):
    k = 2**i
    matrices = bitonic_matrices(k)
    print(f"Testing sorting for {k} elements")
    for j in range(100):
        test = np.random.randint(0, 200, k)
        assert (np.allclose(bisort(matrices, test), np.sort(test)))

Testing sorting for 2 elements
Testing sorting for 4 elements
Testing sorting for 8 elements
Testing sorting for 16 elements
Testing sorting for 32 elements
Testing sorting for 64 elements
Testing sorting for 128 elements
Testing sorting for 256 elements
Testing sorting for 512 elements
Testing sorting for 1024 elements


## Differentiable sorting test

In [5]:
# Differentiable sorting 
np.set_printoptions(precision=2)
matrices = bitonic_matrices(8) 


for i in range(10):
    test = np.random.randint(-200,200,8)
    print("Differentiable", neat_vec(diff_bisort(matrices, test)))
    print("Exact sorting ", neat_vec(bisort(matrices, test)))
    print()

Differentiable -195.00	-183.00	-112.00	-105.00	-75.00	-61.00	-44.00	81.00
Exact sorting  -195.00	-183.00	-112.00	-105.00	-75.00	-61.00	-44.00	81.00

Differentiable -134.00	-20.00	14.87	17.13	140.00	159.00	193.99	199.01
Exact sorting  -134.00	-20.00	15.00	17.00	140.00	159.00	194.00	199.00

Differentiable -184.00	-60.00	-34.00	-3.00	63.00	93.00	131.00	172.00
Exact sorting  -184.00	-60.00	-34.00	-3.00	63.00	93.00	131.00	172.00

Differentiable -182.31	-180.69	-114.00	-55.00	144.00	156.00	170.87	173.13
Exact sorting  -182.00	-181.00	-114.00	-55.00	144.00	156.00	171.00	173.00

Differentiable -199.00	-174.00	-136.00	-81.00	120.00	138.00	146.00	193.00
Exact sorting  -199.00	-174.00	-136.00	-81.00	120.00	138.00	146.00	193.00

Differentiable -194.00	-166.00	-156.00	-58.00	-46.00	75.00	111.00	165.00
Exact sorting  -194.00	-166.00	-156.00	-58.00	-46.00	75.00	111.00	165.00

Differentiable -162.00	-105.00	-61.00	-33.00	-3.01	2.01	31.00	85.00
Exact sorting  -162.00	-105.00	-61.00	-33.00	-3.00	2.00	31

# Relaxed sorting
We can define a slighly modified function which interpolates between `softmax(a,b)` and `mean(a,b)`. The result is a sorting function that can be relaxed from sorting to averaging.

In [6]:
from differentiable_sorting import diff_bisort_smooth
# Differentiable smoothed sorting 
test = np.random.randint(-200,200,8)
print(f"Mean {np.mean(test):.2f}")
print()
print("Exact sorting       ", neat_vec(bisort(matrices, test)))
for smooth in np.linspace(0, 1, 8):    
    print(f"Diff. smooth[{smooth:.2f}]  ", neat_vec(diff_bisort_smooth(matrices, test, smooth)))
        

Mean 53.75

Exact sorting        -87.00	-46.00	43.00	50.00	91.00	103.00	127.00	149.00
Diff. smooth[0.00]   -87.00	-46.00	43.00	50.00	91.00	103.00	127.00	149.00
Diff. smooth[0.14]   -29.52	-7.43	43.43	49.32	79.58	80.70	103.09	110.82
Diff. smooth[0.29]   9.03	19.46	45.85	49.95	66.85	69.81	83.56	85.48
Diff. smooth[0.43]   33.12	36.99	48.84	51.09	59.50	61.89	68.93	69.64
Diff. smooth[0.57]   45.69	46.65	50.22	50.95	56.65	57.61	60.86	61.36
Diff. smooth[0.71]   51.22	51.55	51.76	52.08	55.05	55.38	56.33	56.64
Diff. smooth[0.86]   53.11	53.25	53.26	53.41	54.07	54.22	54.26	54.41
Diff. smooth[1.00]   53.75	53.75	53.75	53.75	53.75	53.75	53.75	53.75


In [7]:
from autograd import jacobian
# show that we can take the derivative
jac_sort = jacobian(diff_bisort_smooth, argnum=1)
jac_sort(matrices, test, 0.05) # slight relaxation

array([[0.02, 0.  , 0.02, 0.02, 0.02, 0.02, 0.02, 0.86],
       [0.86, 0.02, 0.02, 0.02, 0.02, 0.02, 0.  , 0.02],
       [0.02, 0.02, 0.86, 0.02, 0.  , 0.02, 0.02, 0.02],
       [0.02, 0.02, 0.02, 0.  , 0.02, 0.86, 0.02, 0.02],
       [0.02, 0.86, 0.02, 0.05, 0.02, 0.02, 0.  , 0.  ],
       [0.02, 0.05, 0.02, 0.86, 0.  , 0.  , 0.02, 0.02],
       [0.  , 0.  , 0.02, 0.02, 0.04, 0.02, 0.86, 0.02],
       [0.02, 0.02, 0.  , 0.  , 0.86, 0.02, 0.04, 0.02]])

## Woven form
We can "weave" the four matrices into two matrices for fewer multiplies at the cost of having to split and join the matrices at each layer.

In [8]:
from differentiable_sorting import bisort_woven_matrices

def diff_bisort_weave(matrices, x):
    """
    Given a set of bitonic sort matrices generated by bitonic_woven_matrices(n), sort 
    a sequence x of length n.
    """
    split = len(x) // 2
    for weave, unweave in matrices:
        woven = weave @ x
        x = unweave @ np.concatenate(softcswap(woven[:split], woven[split:]))
    return x


woven_matrices = bisort_woven_matrices(8)

print("Exact sorting       ", neat_vec(bisort(matrices, test)))
print(f"Diff. (std.)       ", neat_vec(diff_bisort(matrices, test)))
print(f"Diff. (woven)      ", neat_vec(diff_bisort_weave(woven_matrices, test)))
        

Exact sorting        -87.00	-46.00	43.00	50.00	91.00	103.00	127.00	149.00
Diff. (std.)        -87.00	-46.00	43.00	50.00	91.00	103.00	127.00	149.00
Diff. (woven)       -87.00	-46.00	43.00	50.00	91.00	103.00	127.00	149.00


## Differentiable ranking / argsort
We can use a differentiable similarity measure between the input and output of the vector, e.g. an RBF kernel. We can use this to generate a normalised similarity matrix and apply this to a vector `[1, 2, 3, ..., n]`. This gives a differentiable ranking function.

As `sigma` gets larger, the result converges to giving all values the mean rank; as it goes to zero the result converges to the true rank.

In [9]:
from differentiable_sorting import order_matrix, diff_argsort

In [10]:
matrices = bitonic_matrices(8)

In [11]:
x = [5.0, -1.0, 9.5, 13.2, 16.2, 10.5, 42.0, 18.0]
np.set_printoptions(suppress=True)
print(x)
# show argsort
ranks = diff_argsort(matrices, x, sigma=0.1)
print(neat_vec(ranks))

[5.0, -1.0, 9.5, 13.2, 16.2, 10.5, 42.0, 18.0]
1.00	0.00	2.00	5.00	3.00	4.00	7.00	6.00


In [12]:
# we now have differentiable argmax and argmin by indexing the rank vector
print(np.argmin(x), int(ranks[0]))
print(np.argmax(x), int(ranks[-1]))

1 1
6 6


In [13]:
print("Smoothed ranks")
test = x
for sigma in [0.1, 1, 10, 100, 1000]:     
    ranks = diff_argsort(matrices, test, sigma=sigma) 
    print(f"sigma={sigma:7.1f}  |", neat_vec(ranks))

Smoothed ranks
sigma=    0.1  | 1.00	0.00	2.00	5.00	3.00	4.00	7.00	6.00
sigma=    1.0  | 1.00	0.00	2.89	4.01	3.06	4.42	6.61	6.00
sigma=   10.0  | 2.14	2.70	3.09	3.24	3.46	3.71	3.89	5.91
sigma=  100.0  | 3.46	3.48	3.48	3.49	3.49	3.50	3.50	3.55
sigma= 1000.0  | 3.50	3.50	3.50	3.50	3.50	3.50	3.50	3.50


In [21]:
np.set_printoptions(precision=3)
jac_rank = jacobian(diff_argsort, argnum=1)
print(jac_rank(matrices, np.array(test), 1.0) )

[[ 0.    -0.    -0.    -0.    -0.    -0.    -0.    -0.   ]
 [ 0.     0.    -0.     0.     0.    -0.     0.     0.   ]
 [ 0.004  0.     0.655  0.031  0.002 -0.692  0.     0.   ]
 [ 0.003  0.    -0.64   0.113  0.003  0.522  0.     0.   ]
 [-0.    -0.    -0.007 -0.07  -0.038  0.116 -0.    -0.001]
 [ 0.     0.     0.003 -0.011  0.605  0.007  0.    -0.603]
 [ 0.     0.     0.     0.004 -0.576  0.001  0.     0.571]
 [-0.    -0.    -0.    -0.    -0.    -0.    -0.     0.   ]]


# PyTorch example
We can verify that this is both parallelisable on the GPU and fully differentiable.

In [ ]:
import torch
import numpy as np
from torch.autograd import Variable
import torch.nn.functional as F
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print('Device:', device)

In [ ]:
from differentiable_sorting_torch import diff_bisort, bitonic_matrices, diff_argsort
matrices = bitonic_matrices(16)
torch_matrices = [[torch.from_numpy(matrix).float().to(device) for matrix in matrix_set] for matrix_set in matrices]


In [ ]:
test_input = np.random.normal(0, 5, 16)
var_test_input = Variable(torch.from_numpy(test_input).float().to(device),
                          requires_grad=True)

result = diff_bisort(torch_matrices, var_test_input)

# compute the Jacobian of the sorting function, to show we can differentiate through the
# sorting function
jac = []
for i in range(len(result)):
    jac.append(
        torch.autograd.grad(result[i], var_test_input, retain_graph=True)[0])

# 16 x 16 jacobian of the sorting matrix
print(torch.stack(jac))

In [ ]:
result = diff_argsort(torch_matrices, var_test_input)
print(result)
